<a href="https://colab.research.google.com/github/Pardhu156/Credit-Card-Fraud/blob/main/Credit_Card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

'2.19.0'

## Part 1 - Data Preprocessing

In [ ]:
dataset = pd.read_csv('creditcard.csv')
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
dataset.shape

(284807, 31)

In [ ]:
dataset.isnull().sum().sum()

np.int64(0)

In [ ]:
dataset.select_dtypes(include=['object']).columns

Index([], dtype='object')

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
X.shape, y.shape

((284807, 30), (284807,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train[:, [0, 29]] = sc_X.fit_transform(X_train[:, [0, 29]])
X_test[:, [0, 29]] = sc_X.transform(X_test[:, [0, 29]])

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("Before SMOTE:", np.bincount(y_train))
print("After SMOTE:", np.bincount(y_train_res))

Before SMOTE: [227451    394]
After SMOTE: [227451 227451]


## Part 2 - Implementing ANN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import EarlyStopping

ann = Sequential()

ann.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
ann.add(Dropout(0.2))

ann.add(Dense(16, activation='relu'))
ann.add(Dropout(0.2))

ann.add(Dense(8, activation='relu'))

ann.add(Dense(1, activation='sigmoid'))

ann.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[Precision(), Recall(), AUC()]
)

early = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = ann.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[early]
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - auc: 0.7401 - loss: 0.0450 - precision: 0.1594 - recall: 0.2578 - val_auc: 0.9273 - val_loss: 0.0037 - val_precision: 0.8481 - val_recall: 0.8072
Epoch 2/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - auc: 0.9465 - loss: 0.0038 - precision: 0.7928 - recall: 0.7210 - val_auc: 0.9511 - val_loss: 0.0034 - val_precision: 0.8481 - val_recall: 0.8072
Epoch 3/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - auc: 0.9386 - loss: 0.0038 - precision: 0.8733 - recall: 0.7644 - val_auc: 0.9449 - val_loss: 0.0032 - val_precision: 0.8481 - val_recall: 0.8072
Epoch 4/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - auc: 0.9335 - loss: 0.0037 - precision: 0.8551 - recall: 0.7350 - val_auc: 0.9511 - val_loss: 0.0030 - val_precision: 0.8816 - val_recall: 0.8072
Epoch 5/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - auc: 0.9623 - loss: 0.0030 - precision: 0.8620 - recall: 0.7694 - val_auc: 0.9215 - val_loss: 0.0036 - val_precision: 0.8590 -

In [ ]:
y_pred=ann.predict(X_test)
y_pred=y_pred>0.3

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc = accuracy_score(y_test, y_pred)

prec = precision_score(y_test, y_pred)

rec = recall_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred)

print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1 Score :", f1)

Accuracy : 0.9993504441557529
Precision: 0.7961165048543689
Recall   : 0.8367346938775511
F1 Score : 0.8159203980099502


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[56843    21]
 [   16    82]]
